# Speedtest Tar VS MongoDB

## import libraries & connect MongoDB

In [1]:
import time
import tarfile
import json
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']

import pymongo
from pymongo import MongoClient
client = MongoClient(ip, int(port))

# connect to database and get collections' names
db_twitter = client["DPC"]

## Supporting Methods

In [2]:
# write into csv file
def write_csv(file_name,hashtag,user_location):
    # avoid user location splitted by comma
    try:
        user_location = ''.join(user_location.split(','))
    except:
        pass
    row = "{},{}\n".format(hashtag,user_location)
    
    with open(file_name, 'a') as f:      
        f.write(row)

In [3]:
def get_value(i):
    user_location = i['user']['location']

    if len(i['entities']['hashtags']) == 0:
        hashtag = None
        write_csv(file_name,hashtag,user_location)
    elif len(i['entities']['hashtags']) == 1:
        hashtag = i['entities']['hashtags'][0]['text']
        write_csv(file_name,hashtag,user_location)
    else:
        for h in range(len(i['entities']['hashtags'])):
            hashtag = i['entities']['hashtags'][h]['text']
            write_csv(file_name,hashtag,user_location)

# Tar Test:

In [4]:
start = time.time()

tar = tarfile.open("test.tar.gz", 'r|gz')

# get content of the tar file
for tar_info in tar:
    if tar_info.isfile():
            if tar_info.name.lower().endswith('.json') \
            and r'/.' not in tar_info.name:
                content = tar.extractfile(tar_info).read().decode('utf-8')
                c= json.loads(content)

file_name = "Twitter_2015_tar.csv"
with open(file_name, 'a') as f:
    f.write('hashtag,user_location\n')

for i in c:
    get_value(i)
    
end = time.time()
print("Time cost for tar: " + str(end-start))

print("Tar: csv file for collection is done.")
print("-----------------------")

Time cost for tar: 26.177585124969482
Tar: csv file for collection is done.
-----------------------


# MongoDB Test :

In [5]:
start = time.time()

file_name = "Twitter_2015_mongo.csv"
with open(file_name, 'a') as f:
    f.write('hashtag,user_location\n')

for document in db_twitter["Twitter_2015"].find():
    get_value(document)
    
end = time.time()
print("Time cost for mongo: " + str(end-start))

print("Mongodb: csv file for collection is done.")
print("-----------------------")

4.988497972488403
Time cost for mongo: 4.988497972488403
Mongodb: csv file for collection is done.
-----------------------
